In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
from tokenize_function import tokenize

In [2]:
NnL = pd.read_excel('1.Dataset_files/CharacterData.xlsx',index_col=0)

In [3]:
[(i, NnL['Name'].iloc[i]) for i in range(len(NnL))]

[(0, 'Vernon Dursley'),
 (1, 'Petunia Dursley'),
 (2, 'Dudley Dursley'),
 (3, 'Lily Potter'),
 (4, 'James Potter'),
 (5, 'Harry Potter'),
 (6, 'Violet-cloaked wizard'),
 (7, 'Lord Voldemort'),
 (8, 'Jim McGuffin'),
 (9, 'Ted'),
 (10, "Vernon Dursley's secretary"),
 (11, 'Albus Dumbledore'),
 (12, 'Minerva McGonagall'),
 (13, 'Dedalus Diggle'),
 (14, 'Poppy Pomfrey'),
 (15, 'Rubeus Hagrid'),
 (16, 'Sirius Black'),
 (17, 'Marge Dursley'),
 (18, 'Arabella Figg'),
 (19, 'Tibbles'),
 (20, 'Snowy'),
 (21, 'Mr Paws'),
 (22, 'Tufty'),
 (23, 'Yvonne'),
 (24, 'Piers Polkiss'),
 (25, "Piers Polkiss's mother"),
 (26, 'Keeper of the reptile house'),
 (27, 'Zoo director'),
 (28, 'A boa constrictor'),
 (29, 'A gorilla'),
 (30, 'Dennis'),
 (31, 'Malcolm'),
 (32, 'Gordon'),
 (33, "Dudley's parrot"),
 (34, "Dudley's tortoise"),
 (35, 'Owner of the Railview Hotel'),
 (36, 'Merlin'),
 (37, 'Mr Evans'),
 (38, 'Mrs Evans'),
 (39, 'McKinnon family'),
 (40, 'Bones family'),
 (41, 'Prewett family'),
 (42, 'Cor

In [54]:
texts = np.zeros(len(NnL), dtype=object)
tokens = np.zeros(len(NnL), dtype=object)
aliases = np.zeros(len(NnL), dtype=object)

for char_idx in tqdm(range(len(NnL))):
    query = requests.get(f"https://harrypotter.fandom.com/api.php?action=parse&page={NnL['Link'].iloc[char_idx].split('/')[-1].split('#')[0]}&format=json").json()
    HTML = query['parse']['text']['*']
    soup = BeautifulSoup(HTML, 'html.parser')

    # Get aliases
    try:
        # Character has aliases
        alias_ = soup.find(string='Also known as').findNext("div")
        if alias_.find_all('li') != []:
            # Character has multiple aliases
            alias_ = [i.get_text() for i in alias_.find_all('li')]
            alias_ = [re.split('\s?(\()|(\[)',j)[0] for j in alias_]
        else:
            # Character has only one alias
            alias_ = [re.split('\s?(\()|(\[)',alias_.get_text())[0]]
    except:
        # Character has no aliases
        alias_ = []

    # Ensure that aliases are not the same as a character name
    alias_ = [i for i in alias_ if not NnL['Name'].str.contains(f'{i}', regex=False).any()]
    
    # Add first name to aliases
    none_aliases = ['A', 'Mr', 'Mrs', 'Dr', 'Manager', 'The', 'Father','Sorting', 'wizard', 'Chancellor', 'Workmen', 'Waitress'
                    'Nearly-Headless', 'Fat','Aged', 'Blood-Sucking', 'Forbidden','Unidentified', 'Zoo', 'Kepper', 'Muggle', 'Muggle-Born',
                    'Senior', 'Junior', 'Board', 'Committee','Academy', 'Ministry', 'Department',
                    'Little', 'Great', 'Old', 'Young', 'Head', 'Headmaster', 'Headmistress','Weird','Care', 'Montgomery']
    if ((NnL['Name'].iloc[char_idx].split(' ')[0] not in none_aliases) and
    ("'s" not in NnL['Name'].iloc[char_idx]) and 
    ("s'" not in NnL['Name'].iloc[char_idx]) and
    ("family" not in NnL['Name'].iloc[char_idx]) and
    (NnL['Name'].iloc[char_idx].split(' ')[0].isalpha())):
        alias_.append(NnL['Name'].iloc[char_idx].split(' ')[0])

    # Ensure that aliases are unique and not the same as a character name
    alias_ = list(set(alias_))
    alias_ = [i for i in alias_ if i != NnL['Name'].iloc[char_idx]]

    # Save aliases
    aliases[char_idx] = alias_

    temp = [i.get_text() for i in soup.find_all('p', class_ = None)]

    # Filter out unwanted text
    if 'At least some content in this article' in temp[0]:
        temp = temp[1:]
    if 'At least some content in this article' in temp[0]:
        temp = temp[1:]
    if 'The title of this article is conjectural.' in temp[0]:
        temp = temp[1:]
    if 'This entry needs to be cleaned up to conform to a higher standard of quality.' in temp[0]:
        temp = temp[1:]
    if 'The topic of this article is of a real-life subject' in temp[0]:
        temp = temp[1:]
    if 'The title of this article intentionally uses incorrect spelling or grammar,' in temp[0]:
        temp = temp[1:]
    if 'This page "shows not" an image, but just a written article.' in temp[0]:
        temp = temp[1:]
    if 'Point me!' in temp[0]:
        temp = temp[1:]
    if 'Point me!' in temp[0]:
        temp = temp[1:]
    if 'At least some content in this article' in temp[0]:
        temp = temp[1:]
    if 'The title of this article is conjectural.' in temp[0]:
        temp = temp[1:]
    if 'Biographical information' in temp[0]:
        temp = temp[1:]

    # Get text    
    texts[char_idx] = ' '.join(temp)
    tokens[char_idx] = tokenize(texts[char_idx])

# Save
NnL['WikiText'] = texts
NnL['Tokens'] = tokens
NnL['Aliases'] = aliases

100%|██████████| 707/707 [11:18<00:00,  1.04it/s] 


In [55]:
NnL

,Name,Link,WikiText,Tokens,Aliases
0,Vernon Dursley,/wiki/Vernon_Dursley,"Vernon Dursley was an English Muggle, husband ...","[vernon, dursley, english, muggle, husband, pe...",[Vernon]
1,Petunia Dursley,/wiki/Petunia_Dursley,Petunia Dursley (née Evans) (pre 1960 – pre 20...,"[petunia, dursley, née, evans, pre, pre, engli...","[Petunia, Tuney]"
2,Dudley Dursley,/wiki/Dudley_Dursley,Dudley Dursley (b. 23 June 1980)[1] was the Mu...,"[dudley, dursley, b, june, muggle, son, vernon...","[Ickle Dudleykins, My Neffy Poo, Dudley, Dinky..."
3,Lily Potter,/wiki/Lily_J._Potter,Lily J.[8] Potter (née Evans) (30 January[1] 1...,"[lily, j, potter, née, evans, january, october...",[Lily]
4,James Potter,/wiki/James_Potter_I,"England,[2] Great Britain Godric's Hollow, Wes...","[england, great, britain, godric, hollow, west...","[Prongs, James]"
...,...,...,...,...,...
702,Hugo Weasley,/wiki/Hugo_Granger-Weasley,Hugo Granger-Weasley was born around 2008 to a...,"[hugo, born, around, father, mother, making, t...",[Hugo]
703,Scorpius Malfoy,/wiki/Scorpius_Malfoy,Scorpius Hyperion Malfoy (b. c. 2006) was a Br...,"[scorpius, hyperion, malfoy, british, wizard, ...","[Scorpion King, Malfoy the Unanxious, Scorpius..."
704,Victoire Weasley,/wiki/Victoire_Weasley,Victoire Weasley (b. 2 May[1] 2000 or 2001)[2]...,"[victoire, weasley, b, may, witch, eldest, chi...",[Victoire]
705,Astoria Greengrass,/wiki/Astoria_Malfoy,Great Britain Astoria Malfoy[8] (née Greengras...,"[great, britain, astoria, malfoy, née, greengr...",[Astoria]


In [56]:
[print(i) for i in NnL['Aliases']]

['Vernon']
['Petunia', 'Tuney']
['Ickle Dudleykins', 'My Neffy Poo', 'Dudley', 'Dinky Diddydums', "Big 'D'", 'Popkin', 'Diddykins', 'Dudders', 'Diddy']
['Lily']
['Prongs', 'James']
['Lightning', 'Plotter', 'Harry', 'The Boy Who Lived', 'Vernon Dudley', 'Potty', 'The Boy Who Lied', 'The Chosen One', 'Undesirable No. 1']
[]
['The Dark Lord', 'Lord', 'Heir of Slytherin', 'He-Who-Must-Not-Be-Named', 'Chief Death Eater', 'You-Know-Who']
['Jim']
[]
[]
['Albus']
['Minerva']
['Dedalus']
['Poppy']
['Rubeus', 'Hagger']
['Sirius', 'Padfoot', 'Snuffles']
['Auntie Marge', 'Aunt Marge', 'Marge', 'Margie']
['Arabella']
[]
[]
[]
[]
[]
['Piers']
[]
['Keeper']
[]
[]
[]
[]
[]
[]
[]
[]
['Owner']
['Prince of Enchanters']
[]
[]
[]
[]
[]
['Goblin-Crusher', 'The Other Minister', 'Cornelius']
['Miranda']
['Bathilda', '"Batty" Bagshot']
['Adalbert']
['Emeric']
['Phyllida']
['Arsenus']
['Newton', 'Theseus Scamander']
['Quentin']
[]
['Doris']
['Quirinus']
['Gringotts']
[]
['Madam']
['Draco']
['Lucius']
['Narcissa

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [57]:
NnL.to_csv('CharacterWikis.csv',index=False)